<a href="https://colab.research.google.com/github/Nikhileswar-Komati/Suicide_Ideation/blob/master/3_models_3032.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [394]:
import numpy as np
import pandas as pd
import re
from collections import defaultdict
np.seterr(divide = 'ignore')

{'divide': 'ignore', 'invalid': 'warn', 'over': 'warn', 'under': 'ignore'}

In [ ]:
from google.colab import files
files.upload()

In [396]:
data = pd.read_csv('/content/Suicide_Watch_PRAW_3032.csv')
data.head(5)

,title,id,subreddit,body
0,New wiki on how to avoid accidentally encourag...,cz6nfd,SuicideWatch,We've been seeing a worrying increase in pro-s...
1,Please remember that NO ACTIVISM of any kind i...,iq0w21,SuicideWatch,"Activism, i.e. advocating or fundraising for s..."
2,Failed suicide = Expensive,kajy93,SuicideWatch,I recently made a suicide attempt. Unfortunate...
3,Being alive is pointless (makes no sense),kay9t0,SuicideWatch,Let's say you're mentally healthy and not suic...
4,My cat is the only reason I'm still alive.,kaz14g,SuicideWatch,Let's start saying that I'm Italian so my engl...


In [397]:
data.shape

(3032, 4)

In [398]:
data['subreddit'].value_counts()

SuicideWatch    2011
depression      1021
Name: subreddit, dtype: int64

In [399]:
data['body'][0]

'We\'ve been seeing a worrying increase in pro-suicide content showing up here and, and also going unreported. This undermines our purpose here, so we wanted to highlight and clarify our guidelines about both direct and indirect incitement of suicide.  \n\nWe\'ve created a wiki that covers these issues.  We hope this will be helpful to anyone who\'s wondering whether something\'s okay here and which responses to report.  It explains in detail why *any* validation of suicidal intent, even an "innocent" message like "if you\'re 100% committed, I\'ll just wish you peace" is likely to increase people\'s pain, and why it\'s important to report even subtle pro-suicide comments. The full text of the wiki\'s current version is below, and it is maintained at [/r/SuicideWatch/wiki/incitement](http://www.reddit.com/r/SuicideWatch/wiki/incitement). \n\nWe deeply appreciate everyone who gives responsive, empathetic, non-judgemental support to our OPs, and we particularly thank everyone who\'s alrea

In [400]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [401]:
text = "Hello, Stupid:: idiot!"
def preprocess(string):
  string = str(string)
  phrase = re.sub(r"n\'t", " not", string)
  phrase = re.sub(r"\'re", " are", phrase)
  phrase = re.sub(r"\'s", " is", phrase)
  phrase = re.sub(r"\'d", " would", phrase)
  phrase = re.sub(r"\'ll", " will", phrase)
  phrase = re.sub(r"\'t", " not", phrase)
  phrase = re.sub(r"\'ve", " have", phrase)
  phrase = re.sub(r"\'m", " am", phrase)
  phrase = re.sub('[^a-z0-9]+', ' ', phrase, flags=re.IGNORECASE)
  phrase = re.sub('(\s+)', ' ', phrase)
  phrase = phrase.lower()

  text_tokens = word_tokenize(phrase)
  return " ".join([word for word in text_tokens if not word in stopwords.words()])
print(preprocess(data['body'][0]))

seeing worrying increase pro suicide content showing going unreported undermines purpose wanted highlight clarify guidelines direct indirect incitement suicide created wiki covers issues hope helpful anyone wondering whether something okay responses report explains detail validation suicidal intent even innocent message like 100 committed wish peace likely increase people pain important report even subtle pro suicide comments full text wiki current version maintained r suicidewatch wiki incitement http www reddit r suicidewatch wiki incitement deeply appreciate everyone gives responsive empathetic judgemental support ops particularly thank everyone already reporting incitement forms please report post comment encourages suicide breaks guidelines sidebar moderators either clicking report button sending us modmail https www reddit message compose 2fr 2fsuicidewatch link deal guideline violations reported us soon read everything community reports essential get pm breaks guidelines please 

In [402]:
data['text'] = data['title'] + data['body']
data['text'] = data['text'].map(lambda string: preprocess(string))

In [403]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import logging
logging.basicConfig(level=logging.INFO)
from tensorflow.keras import layers

In [404]:
!wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py

In [405]:
!pip install sentencepiece
!pip install bert-for-tf2

In [406]:
import tensorflow_hub as hub
import bert
import tokenization
module_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2'
bert_layer = hub.KerasLayer(module_url, trainable=True)


INFO:absl:resolver HttpCompressedFileResolver does not support the provided handle.
INFO:absl:resolver GcsCompressedFileResolver does not support the provided handle.


In [407]:
X = data['text'].values
y = data['subreddit'].values

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

y = le.fit_transform(y)
# X = list(X)
# y = list(y)
print(set(y))
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size = 0.02, random_state = 42)
print(len(train_X), len(test_X))

{0, 1}
2971 61


In [408]:
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence) + [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [409]:
def build_model(bert_layer, max_len=512):
    input_word_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    clf_output = sequence_output[:, 0, :]
    net = tf.keras.layers.Dense(32, activation='relu')(clf_output)
    net = tf.keras.layers.Dropout(0.5)(net)
    # net = tf.keras.layers.Dense(32, activation='relu')(net)
    # net = tf.keras.layers.Dropout(0.5)(net)
    out = tf.keras.layers.Dense(1, activation='sigmoid')(net)
    
    model = tf.keras.models.Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(tf.keras.optimizers.Adam(lr=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [410]:
import keras
max_len = 100
train_input = bert_encode(train_X, tokenizer, max_len = max_len)
test_input = bert_encode(test_X, tokenizer, max_len = max_len)
train_labels = train_y

In [411]:
model = build_model(bert_layer, max_len=max_len)
model.summary()

Model: "functional_87"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 100)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 100)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 100)]        0                                            
__________________________________________________________________________________________________
keras_layer_7 (KerasLayer)      [(None, 768), (None, 109482241   input_word_ids[0][0]             
                                                                 input_mask[0][0]     

In [412]:
checkpoint = tf.keras.callbacks.ModelCheckpoint('model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)
earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, verbose=1)

train_history = model.fit(
    train_input, train_labels, 
    validation_split = 0.2,
    epochs = 5,
    callbacks = [checkpoint, earlystopping],
    batch_size = 16,
    verbose = 1)

Epoch 1/5
149/149 [==============================] - ETA: 0s - loss: 0.6468 - accuracy: 0.6397
Epoch 00001: val_accuracy improved from -inf to 0.66218, saving model to model.h5
149/149 [==============================] - 66s 440ms/step - loss: 0.6468 - accuracy: 0.6397 - val_loss: 0.5999 - val_accuracy: 0.6622
Epoch 2/5
149/149 [==============================] - ETA: 0s - loss: 0.5580 - accuracy: 0.7256
Epoch 00002: val_accuracy improved from 0.66218 to 0.73445, saving model to model.h5
149/149 [==============================] - 65s 437ms/step - loss: 0.5580 - accuracy: 0.7256 - val_loss: 0.5339 - val_accuracy: 0.7345
Epoch 3/5
149/149 [==============================] - ETA: 0s - loss: 0.4765 - accuracy: 0.7891
Epoch 00003: val_accuracy did not improve from 0.73445
149/149 [==============================] - 59s 394ms/step - loss: 0.4765 - accuracy: 0.7891 - val_loss: 0.5646 - val_accuracy: 0.7008
Epoch 4/5
149/149 [==============================] - ETA: 0s - loss: 0.3973 - accuracy: 0.8

In [413]:
model.load_weights('model.h5')
test_pred = model.predict(test_input)

In [414]:
def my_round(val):
  if val >= 0.5:
    return 1
  return 0
test_pred = [my_round(ele) for ele in test_pred]

In [415]:
from sklearn.metrics import accuracy_score
acc = accuracy_score(test_y, test_pred)
print ("Test Set Examples: ", len(test_y)) 
print ("Test Set Accuracy: ", acc * 100, "%")

Test Set Examples:  61
Test Set Accuracy:  77.04918032786885 %


In [416]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier
clf = Pipeline([('cv', CountVectorizer()), ('xgb', XGBClassifier())])

clf.fit(train_X, train_y)
print("------------Training Done ----------")
predictions = clf.predict(test_X)

test_acc_sklearn = np.sum(predictions == test_y) / float(len(test_y)) 


print ("Test Set Examples: ", len(test_y)) 
print ("Test Set Accuracy: ", test_acc_sklearn * 100, "%")

------------Training Done ----------
Test Set Examples:  61
Test Set Accuracy:  68.85245901639344 %


In [417]:
vocab_size = 10000
embedding_dim = 50
max_length = 100
oov_tok = '<OOV>'
trunc_type = 'post'

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token = oov_tok)
tokenizer.fit_on_texts(train_X)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(train_X)
padded = pad_sequences(sequences, maxlen = max_length, truncating = trunc_type)

testing_sequences = tokenizer.texts_to_sequences(test_X)
testing_padded = pad_sequences(testing_sequences, maxlen = max_length)

In [418]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 100, 50)           500000    
_________________________________________________________________
flatten_5 (Flatten)          (None, 5000)              0         
_________________________________________________________________
dense_134 (Dense)            (None, 32)                160032    
_________________________________________________________________
dense_135 (Dense)            (None, 1)                 33        
Total params: 660,065
Trainable params: 660,065
Non-trainable params: 0
_________________________________________________________________


In [419]:
checkpoint = tf.keras.callbacks.ModelCheckpoint('model2.h5', monitor='val_accuracy', save_best_only=True, verbose=1)
earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, verbose=1)

num_epochs = 10
model.fit(padded, train_y, 
          epochs = num_epochs, 
          validation_data = (testing_padded, test_y),
          callbacks = [checkpoint, earlystopping],
          batch_size = 16,
          verbose = 1)

Epoch 1/10
184/186 [============================>.] - ETA: 0s - loss: 0.6164 - accuracy: 0.6695
Epoch 00001: val_accuracy improved from -inf to 0.65574, saving model to model2.h5
186/186 [==============================] - 1s 7ms/step - loss: 0.6152 - accuracy: 0.6705 - val_loss: 0.5754 - val_accuracy: 0.6557
Epoch 2/10
184/186 [============================>.] - ETA: 0s - loss: 0.3039 - accuracy: 0.8920
Epoch 00002: val_accuracy did not improve from 0.65574
186/186 [==============================] - 1s 7ms/step - loss: 0.3025 - accuracy: 0.8926 - val_loss: 0.6654 - val_accuracy: 0.6066
Epoch 3/10
182/186 [============================>.] - ETA: 0s - loss: 0.0792 - accuracy: 0.9797
Epoch 00003: val_accuracy improved from 0.65574 to 0.68852, saving model to model2.h5
186/186 [==============================] - 1s 6ms/step - loss: 0.0780 - accuracy: 0.9801 - val_loss: 0.6298 - val_accuracy: 0.6885
Epoch 4/10
181/186 [============================>.] - ETA: 0s - loss: 0.0471 - accuracy: 0.9890